# Export the learned weights as an NPZ

The aim here is to be able to use the pretrained weights in other frameworks (like tensorflow)

In [ ]:
%pylab inline

In [ ]:
import caffe
caffe.set_mode_cpu() #  No need to burden the GPU with this

NOTE: We should use [BN-absorber.py](https://github.com/TimoSaemann/caffe-segnet-cudnn5/blob/master/scripts/BN-absorber.py) first perhaps?

In [ ]:
proto = 'modified-inference-net.prototxt'
weights = 'deploy/test_weights.caffemodel'
filename = 'weights.npz'

In [ ]:
net = caffe.Net(proto, weights, caffe.TEST)

In [ ]:
result = {}
for param in net.params:
    blobs = net.params[param]
    if 'bn' in str(param):
        result['{}/{}'.format(param.replace('-','_'), 'gamma')] = blobs[0].data
        result['{}/{}'.format(param.replace('-','_'), 'beta')] = blobs[1].data
    else:
        result['{}/{}'.format(param.replace('-','_'), 'weights')] = blobs[0].data
        result['{}/{}'.format(param.replace('-','_'), 'biases')] = blobs[1].data

In [ ]:
np.savez_compressed(filename, **result)

In [ ]:
for k in result:
    print '{:32} mu:{:+07.4f}, std:{:+07.4f}'.format(k.ljust(32,'.'), result[k].mean(), result[k].std())

In [ ]:
#Test
npfile = np.load(filename)

for k in npfile:
    print '{:32}{}'.format(k.ljust(32,'.'), np.allclose(npfile[k], result[k]))
npfile.close()